# On-policy Prediction with Approximation

Approximate value function $v_\pi$ with $\hat{v}(s, \mathbf{w}) \approx v_\pi(s)$, where $\mathbf{w} \in \mathbb{R}^d$, and $d \ll |\mathscr{S}|$. Because we cannot memorize a value for every state, the approximate value function must learn to _generalize_ and learn the value of previously unseen states.

Learning a value function makes RL techniques also apply to partially observable problems (POMDPs), since the features available to the value function may only consist of the observable features.

However, a value function approximator does not guarantee that the history of the state is accounted for.

# 9.1 Value-function Approximation

Individual update $s \mapsto u$. Use supervised learning using update targets $\{ s_i \mapsto u_i \}$ as a dataset.

Must be able to learn incrementally, and handle a moving target since $\pi$ changes with $q_\pi$

# 9.2 The Prediction Objective ($\mathrm{\overline{VE}}$)

Value function updates affect the prediction for multiple states.

It may not be possible to exactly predict the value of every state.

Assuming undercapacity to learn exact value function.

Specify a distribution over states $\mu(s)$ $\sum_s{\mu(s)} = 1$


_Mean squared value error_ : $\mathrm{\overline{VE}}$

$$
\begin{align}
\mathrm{\overline{VE}}(\mathbf{w})& \; \dot{=} \; \mathbb{E}_{S\sim\mu}\left[\left(v_\pi(S) - \hat{v}(S, \mathbf{w})\right)^2\right]\\
&= \sum_{s\in\mathscr{S}}{\mu(s)\left(v_\pi(s) - \hat{v}(s, \mathbf{w})^2\right)}
\end{align}
$$

$\sqrt{\mathrm{\overline{VE}}}$ is a measure of the typical deviation between the true value and the approximated value. $\mu(s)$ is typically chosen to be or to approximate the fraction of time the agent reaches $s$, and is called the _on-policy distribution_.

**The on-policy distribution in episodic tasks**

Let the initial state distribution be $S_0 \sim h$ and $\eta(s)$ be the average number of time steps spent in state $s$ in a single episode.

$$
\eta(s) = h(s) + \sum_{s'\in\mathscr{S}, a\in\mathscr{A}(s')}\eta(s')p(s\mid a, s') \pi(a\mid s')
$$

This can be thought of as a dynamic programming equation for the number of visits to a state $s$. It can be solved and normalized to give $\mu$

$$
\mu(s) = \frac{\eta(s)}{\sum_{s'}{\eta(s')}}
$$

Discounting can be added by multiplying the second term of the recurrence equation for $\eta(s)$ by $\gamma$.

In continuing tasks, the on-policy distribution is the stationary distribution of states under policy $\pi$.

$$
\mu(s) = \sum_{s'\in\mathscr{S},a\in\mathscr{A}(s')}{\mu(s')\pi(a\mid s')p(s \mid s', a)}
$$

Ideally, we would want to find a global optimum for $\mathrm{\overline{VE}}$, that is, a weight vector $\mathbf{w}^*$ such that:

$$
\mathrm{\overline{VE}}(\mathbf{w}^*) \le \mathrm{\overline{VE}}(\mathbf{w}) \quad \mathrm{for\,all} \; \mathbf{w} \in \mathbb{R}^d
$$

This is not always possible for complex hypothesis classes. We can however find _local_ optima for these cases. No guarantee for convergence to a global optimum, or even within a bound of the optimum. Can even diverge.

Not all function approximation schemes can be considered here, so we limit ourselves to gradient based methods, particularly linear ones.

## Stochastic-gradient and Semi-gradient Methods

Consider a fixed size weight vector $\mathbf{w} = \begin{pmatrix} w_1 & w_2 & w_3 & \dots & w_d\end{pmatrix}^T$. Suppose further that $v(s, \mathbf{w})$ is a differentiable function of $\mathbf{w}$ at all states $s$. We will be incrementally updating an estimate of $\mathbf{w}$, starting at some initial estimate $\mathbf{w}_0$, in order to form the sequence $\mathbf{w}_t$ for $t = 0, 1, 2, \dots$. We observe a new sample state $S_t$ on each time step, and it's true value under the current policy. To select $\mathbf{w}$, we perform stochastic gradient descent as follows:

$$
\begin{align}
\mathbf{w}_{t+1} & \; \dot{=} \; \mathbf{w}_t - \frac{1}{2}\alpha\nabla_\mathbf{w}\left[v_\pi(S_t) - \hat{v}(S_t, \mathbf{w_t})\right]^2\\
&= \mathbf{w}_t + \alpha\left(v_\pi(S_t) - \hat{v}(S_t, \mathbf{w}_t)\right)\nabla_{\mathbf{w}}\hat{v}(S_t, \mathbf{w}_t)
\end{align}
$$

where $\nabla_\mathbf{w} f(\mathbf{w}) = \begin{pmatrix} \frac{\partial f}{\partial w_1} & \frac{\partial f}{\partial w_2} & \dots & \frac{\partial f}{\partial w_d} \end{pmatrix}\$

We do not attempt zero out the error on the example since we need to balance other states. Additionally, there is no guarantee that the error on this particular state will be made zero by moving in the gradient direction.

Stochastic descent only converges when the step-size decreases over time such that:

$$
\sum_t{\alpha_t} = \infty \quad \sum_t{\alpha_t^2} = 0
$$

Now consider the case where $v_\pi(s)$ is unavailable, and instead only one of the bootstrap targets, or an otherwise corrupted form of $v_\pi(s)$, is available. We call this value $U_t$ as in the earlier section. By substituting $v_\pi(s)$ with $U_t$, we get the update step for gradient based value estimation:

$$
\mathbf{w}_{t+1} = \mathbf{w}_t + \left(U_t - \hat{v}(S_t, \mathbf{w}_t)\right)\nabla_{\mathbf{w}}{\hat{v}(S_t, \mathbf{w})}
$$

If we select $U_t = G_t$, we get _Gradient Monte-Carlo Value Estimation_.

Note however that if we were to have used a bootstrapped estimate such as:

$$
U_t = R_t + \gamma \hat{v}(S_{t+1}, \mathbf{w})
$$

will depend on the current value of $\mathbf{w}$ and therefore be _biased estimators_ of $v_\pi$, and so convergence is not guaranteed. Therefore, gradient bootstrapping methods are _not_ technically gradient descent methods, as our estimator is not guaranteed to have the true value of the gradient as its expectation. They account only for the participation of $\mathbf{w}$ on the right hand side of the difference, but not for the update in the target. Therefore, they only include part of the gradient and are called _semi-gradient_ methods.

Despite this, semi-gradient methods still converge robustly, can be used on continuing problems, and converge more quickly than Monte-Carlo methods for the reasons discussed in Chapter 6.

### State aggregation

State aggregation assigns one estimated value (and thus a component of $\mathbf{w}$ to an entire collection of states. So long as the partitioning of states is computable, state aggregation can be used with stochastic gradient descent.

**Example: 1000 State Random 

In [28]:
function state_aggregation_gradient_mc(;size=1000, N=1, d=10, α=2e-5)
    𝒮 = 1:size
    bucket_size = size ÷ d
    V = zeros(d)
    for i in 1:N
        S = Int[]
        s = size ÷ 2
        println("iter $(i)")
        while s in 𝒮
            push!(S, s)
            lr = rand() < 0.5 ? -1 : 1
            sp = s + lr * rand(1:100)
            if sp > 1000
                println("End right")
                sp = 1001
                r = 1
            elseif sp < 1
                println("End left")
                sp = 0
                r = -1
            end
            s = sp
        end
            
        for s in reverse(S)
            V[s ÷ bucket_size] += α*(r - V[s ÷ bucket_size])
        end
    end
    
    return V
end

state_aggregation_gradient_mc (generic function with 1 method)

In [29]:
state_aggregation_gradient_mc()

iter 1
End left


LoadError: BoundsError: attempt to access 10-element Vector{Float64} at index [0]

## Linear Methods

We consider the case where $\hat{v}(s, \mathbf{w})$ is linear in $\mathbf{w}$. We assume that for each state there is a feature vector $\mathbf{x}(s) \in \mathbb{R}^d$. We can then define:

$$
\hat{v}(s, \mathbf{w})) = \mathbf{w}^\top\mathbf{x}(s)
$$

In this case the gradient expression is incredibly simple:

$$
\nabla_{\mathbf{w}}v(s, w) = \mathbf{x}(s)
$$

And so we have as our update:

$$
\mathbf{w}_{t+1} = \mathbf{w}_t + \alpha\left(U_t - \mathbf{w}^\top\mathbf{x}(s)\right)\mathbf{x}(s)
$$

Guaranteed to converge to a _global_ optimum

> _Exercise 9.1_ Show that tabular methods such as presented in Part I of this book are a special case of linear function approximation. What would the feature vectors be?

It is a special case of linear function approximation where, for every state $s_i$ 

$$
x_i(s) = \begin{cases} 1 & s = s_i \\ 0 & \mathrm{otherwise} \end{cases}
$$

This works for cases where $|\mathscr{S}| < \infty$.

# 9.5 Feature Construction for Linear Methods

> _Exercise 9.2_ Why does (9.17) define $(n+1)^k$ distinct features for dimension $k$?

It defines $(n+1)^k$ features because a feature will be defined by a selection of exponents on each of the $k$ state variables. Since there are $n + 1$ available exponents (including zero), the total number of features is $(n + 1)^k$

> _Exercise 9.3_ What $n$ and $c_{i,j}$ produce the feature vectors $\mathbf{x}(s) = \begin{pmatrix} 1 & s_1 & s_2 & s_1 s_2 & s_1^2 & s_2^2 & s_1 s_2^2 & s_1^2 s_2 & s_1^2 s_2^2 \end{pmatrix}$

$n = 2$ and

$$
c = \begin{pmatrix} 0 & 0 \\ 1 & 0 \\ 0 & 1 \\ 1 & 1 \\ 2 & 0 \\ 0 & 2 \\ 2 & 2 \end{pmatrix}
$$

# 9.5.4 Tile Coding

> _Exercise 9.4_ Suppose we believe that one of two state dimensions is more likely to have an effect on the value function than is the other, that generalization should be primarily across this dimension than along it. What kind of tilings could be used to take advantage of this prior knowledge?

One could use tiles that are asymmetrically elongated along the dimension that we expect to have less of an effect. This increases the scale of generalization along that dimension and leads to the overall value of that feature being learned more quickly but with less precision.